In [ ]:
import sys
sys.path.append('..')

In [ ]:
!nvidia-smi

In [ ]:
%load_ext autoreload

In [ ]:
import os
os.getcwd()

# Imports

In [1]:
from typing import Any

import matplotlib.pyplot as plt

import os
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torchvision.datasets import MNIST, CIFAR100
from torch.utils.data import DataLoader
import lightning as L
import torch.optim as optim

from PIL import Image
from PIL import Image, ImageOps
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import DeviceStatsMonitor

# Code

In [17]:
# First we need to generate the dataset and dataloader



# function to resize image
def resize_image(src_image, size=(32,32), bg_color="white"):


    # resize the image so the longest dimension matches our target size
    src_image.thumbnail(size, Image.ANTIALIAS)

    # Create a new square background image
    new_image = Image.new("RGB", size, bg_color)

    # Paste the resized image into the center of the square background
    new_image.paste(src_image, (int((size[0] - src_image.size[0]) / 2), int((size[1] - src_image.size[1]) / 2)))

    # return the resized image
    return new_image

def load_dataset(data_path):
    transformers = transforms.Compose(
        [
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]
    )

    full_dataset = ImageFolder(root = data_path, transform=transformers)

    # Split into training (80% and testing (20%) datasets)
    train_size = int(0.8 * len(full_dataset))
    test_size = len(full_dataset) - train_size

    # use torch.utils.data.random_split for training/test split
    train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])


    # define a loader for the training data we can iterate through in 50-image batches
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=64,
        num_workers=4,
        shuffle=False
    )

    # define a loader for the testing data we can iterate through in 50-image batches
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=64,
        num_workers=4,
        shuffle=False
    )

    return train_loader, test_loader


# Visualization

In [ ]:
# train_folder = '../data/tiny-imagenet/tiny-imagenet-200/train'

# # Get the iterative dataloaders for test and training data
# train_loader, test_loader = load_dataset(train_folder)
# batch_size = train_loader.batch_size
# print("Data loaders ready to read", train_folder)

In [ ]:
# x, y = next(iter(train_loader))

# # Randomly select 10 indices
# indices = np.random.choice(range(len(x)), 10, replace=False)

# # Create a subplot for 2 rows and 5 columns
# fig, axs = plt.subplots(2, 5, figsize=(25, 10))

# for i, idx in enumerate(indices):
#     # Select the image and label
#     img = x[idx]
#     label = y[idx]

#     # PyTorch tensors assume (C, H, W) - need to change to (H, W, C) for plotting
#     img = img.permute(1, 2, 0)

#     # Normalize the image to [0, 1] for plotting
#     img = (img - img.min()) / (img.max() - img.min())

#     # Plot the image
#     ax = axs[i // 5, i % 5]
#     ax.imshow(img)
#     ax.axis('off')
#     ax.set_title(f"Label: {label}")

# plt.show()

In [ ]:
# import torch
# model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', weights=None)
# model.eval()

# x, y = next(iter(train_loader))


# # move the input and model to GPU for speed if available
# if torch.cuda.is_available():
#     x = x.to('cuda')
#     model.to('cuda')

# with torch.no_grad():
#     output = model(x)
# # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
# probabilities = torch.nn.functional.softmax(output[0], dim=0)
# print(probabilities.size())
# print(f"{torch.max(probabilities)}")
# print(torch.argmax(probabilities))
# print(f"{y[0] = }")


# Learning Code

In [14]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import os

from tqdm.notebook import tqdm

In [15]:
# Define your AlexNet model
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.alex_net = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=False)

    def forward(self, x):
        return self.alex_net(x)

In [38]:
data_path = "../data/tiny-imagenet/tiny-imagenet-200/train"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

train_loader, val_loader = load_dataset(data_path=data_path)

# Initialize model, loss function, and optimizer
model = AlexNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Using cache found in /home/soroush1/.cache/torch/hub/pytorch_vision_v0.10.0


In [40]:
# Set up TensorBoard
os.makedirs('log', exist_ok=True)
writer = SummaryWriter('log')

overfit_batch = next(iter(train_loader))
overfit_batch = [overfit_batch]
# overfit_batch = 
# overfit_batch = overfit_batch.unsqueeze(0)
# print(f"{overfit_batch.size()}")

# Training loop
num_epochs = 500
for epoch in tqdm(range(num_epochs)):
    model.train()
    train_loss = 0.0
    for inputs, labels in overfit_batch:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # Validation loop
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in overfit_batch:
            print(f"{inputs.size()}")
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    # Calculate average losses and log them
    train_loss /= len(train_loader)
    val_loss /= len(val_loader)
    print(f"Epoch {epoch+1}/{num_epochs} - Train loss: {train_loss:.4f}, Validation loss: {val_loss:.4f}")
    writer.add_scalar('Loss/Train', train_loss, epoch)
    writer.add_scalar('Loss/Validation', val_loss, epoch)

writer.close()

  0%|          | 0/500 [00:00<?, ?it/s]

torch.Size([64, 3, 224, 224])
Epoch 1/500 - Train loss: 0.0055, Validation loss: 0.0155
torch.Size([64, 3, 224, 224])
Epoch 2/500 - Train loss: 0.0040, Validation loss: 0.0215
torch.Size([64, 3, 224, 224])
Epoch 3/500 - Train loss: 0.0054, Validation loss: 0.0200
torch.Size([64, 3, 224, 224])
Epoch 4/500 - Train loss: 0.0050, Validation loss: 0.0158
torch.Size([64, 3, 224, 224])
Epoch 5/500 - Train loss: 0.0040, Validation loss: 0.0146
torch.Size([64, 3, 224, 224])
Epoch 6/500 - Train loss: 0.0037, Validation loss: 0.0143
torch.Size([64, 3, 224, 224])
Epoch 7/500 - Train loss: 0.0036, Validation loss: 0.0142
torch.Size([64, 3, 224, 224])
Epoch 8/500 - Train loss: 0.0036, Validation loss: 0.0139
torch.Size([64, 3, 224, 224])
Epoch 9/500 - Train loss: 0.0035, Validation loss: 0.0137
torch.Size([64, 3, 224, 224])
Epoch 10/500 - Train loss: 0.0034, Validation loss: 0.0136
torch.Size([64, 3, 224, 224])
Epoch 11/500 - Train loss: 0.0034, Validation loss: 0.0134
torch.Size([64, 3, 224, 224])


In [ ]:
print("hello")